In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

class ResNetFPNFeatureExtractor(nn.Module):
    """
    Implements the Feature Extractor from diagram (b).
    Uses a ResNet backbone and FPN-like fusion.
    """
    def __init__(self, backbone_name='resnet18', pretrained=True, 
                 out_channels=256, in_channels=3):
        super().__init__()
        
        # 1. Load ResNet backbone using timm
        self.backbone = timm.create_model(
            backbone_name,
            pretrained=pretrained,
            features_only=True, # Return features from each stage
            in_chans=in_channels,
            out_indices=(1, 2, 3, 4) # We need stages 1, 2, 3, 4
        )
        
        # Get the channel counts from the backbone stages
        backbone_channels = self.backbone.feature_info.channels()
        # Example for resnet18: [64, 128, 256, 512]

        # 2. Create the 1x1 Convs for channel reduction (lateral layers)
        self.lateral_convs = nn.ModuleList()
        for in_c in backbone_channels:
            self.lateral_convs.append(nn.Conv2d(in_c, out_channels, kernel_size=1))
            
        # 3. Create the final fusion convolutions (segmentation head)
        # Input channels = num_stages * out_channels
        self.fusion_convs = nn.Sequential(
            nn.Conv2d(len(backbone_channels) * out_channels, out_channels, 
                      kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, # Keep the same channel dim
                      kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            # The diagram shows one more 1x1 conv in the head, let's add it
            nn.Conv2d(out_channels, out_channels, kernel_size=1)
        )

    def forward(self, x):
        # 1. Get feature maps from ResNet stages
        # This will be a list of tensors, e.g., [C1, C2, C3, C4]
        # C1 is 1/4 res, C2 is 1/8, C3 is 1/16, C4 is 1/32
        features = self.backbone(x)
        
        processed_features = []
        target_size = features[0].shape[2:] # Size of the C1 feature map (1/4 res)

        # 2. Apply 1x1 convs and upsample
        for i, feature_map in enumerate(features):
            lateral_feature = self.lateral_convs[i](feature_map)
            
            # Upsample if not the first (highest-res) feature map
            if i > 0:
                lateral_feature = F.interpolate(lateral_feature, size=target_size, 
                                                mode='bilinear', align_corners=False)
            processed_features.append(lateral_feature)
            
        # 3. Concatenate all processed & upsampled features
        fused_features = torch.cat(processed_features, dim=1)
        
        # 4. Apply final fusion convolutions
        output_feature_map = self.fusion_convs(fused_features)
        
        return output_feature_map # Shape: (B, out_channels, H/4, W/4)

/home/melihkaan/Desktop/CS415PROJECT/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BAM(nn.Module):
    """ Implements the Basic Attention Module (BAM) from diagram (c). """
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels
        
        # 1x1 convolutions to generate Query, Key, Value
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        
        self.softmax = nn.Softmax(dim=-1) # Apply softmax along the key dimension

        # Learnable scaling parameter
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        batch_size, C, H, W = x.size()
        
        # Generate Q, K, V
        # Reshape for matrix multiplication: (B, C', N) where N = H*W
        proj_query = self.query_conv(x).view(batch_size, -1, W * H).permute(0, 2, 1) # B, N, C'
        proj_key = self.key_conv(x).view(batch_size, -1, W * H) # B, C', N
        proj_value = self.value_conv(x).view(batch_size, -1, W * H) # B, C, N
        
        # Calculate attention map: A = softmax(Q @ K)
        energy = torch.bmm(proj_query, proj_key) # B, N, N
        attention = self.softmax(energy) # B, N, N
        
        # Apply attention to Value: out = V @ A.T
        out = torch.bmm(proj_value, attention.permute(0, 2, 1)) # B, C, N
        
        # Reshape back to image format: (B, C, H, W)
        out = out.view(batch_size, C, H, W)
        
        # Apply learnable scaling and add residual connection
        out = self.gamma * out + x
        return out

In [3]:
class PAM(nn.Module):
    """ Implements the Pyramid Attention Module (PAM) from diagram (d). """
    def __init__(self, in_channels):
        super().__init__()
        
        # Define scales for the pyramid (e.g., 1x1, 2x2, 3x3, 6x6 avg pooling)
        self.scales = [1, 2, 3, 6] 
        
        # Create BAM modules for each scale's processing path
        self.bams = nn.ModuleList()
        # Create 1x1 convs to reduce channels before BAM at different scales
        self.convs = nn.ModuleList()
        
        for _ in self.scales:
            # Reduce channels slightly before BAM? (Optional, helps efficiency)
            # Or keep the same channels as input
            processed_channels = in_channels # Keep same for simplicity
            self.convs.append(nn.Sequential(
                nn.Conv2d(in_channels, processed_channels, kernel_size=1, bias=False),
                nn.BatchNorm2d(processed_channels),
                nn.ReLU(inplace=True)
            ))
            self.bams.append(BAM(processed_channels))
            
        # Final fusion layer after concatenating pyramid outputs
        self.fuse_conv = nn.Sequential(
            nn.Conv2d(in_channels + len(self.scales) * in_channels, in_channels, 
                      kernel_size=1, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        batch_size, C, H, W = x.size()
        
        pyramid_features = [x] # Start with the original features
        
        for i, scale in enumerate(self.scales):
            # Apply pooling for different scales
            if scale > 1:
                pooled_x = F.adaptive_avg_pool2d(x, (H // scale, W // scale))
            else:
                pooled_x = x # Scale 1 is the original

            # Process with 1x1 conv and BAM
            processed_x = self.convs[i](pooled_x)
            attended_x = self.bams[i](processed_x)
            
            # Upsample back to original size
            upsampled_x = F.interpolate(attended_x, size=(H, W), mode='bilinear', align_corners=False)
            pyramid_features.append(upsampled_x)
            
        # Concatenate original features and all pyramid features
        fused_features = torch.cat(pyramid_features, dim=1)
        
        # Apply final fusion convolution
        out = self.fuse_conv(fused_features)
        
        # Add residual connection
        return out + x # The diagram shows a final residual connection

In [4]:
class STANet(nn.Module):
    def __init__(self, backbone_name='resnet34', fpn_out_channels=256, 
                 pam_channels=256, classes=1, pretrained=True, in_channels=3):
        super().__init__()
        
        # --- 1. Siamese Feature Extractor ---
        # Create ONE shared instance
        self.feature_extractor = ResNetFPNFeatureExtractor(
            backbone_name=backbone_name,
            pretrained=pretrained,
            out_channels=fpn_out_channels,
            in_channels=in_channels
        )
        
        # --- 2. Spatial-Temporal Attention Module ---
        # Using PAM here as it's more advanced than BAM alone
        # Input channels should match the output of the feature extractor
        self.pam = PAM(in_channels=fpn_out_channels) 
        
        # --- 3. Metric Module (Final Classifier Head) ---
        # Takes the PAM output and produces the final change map
        self.metric_module = nn.Sequential(
            nn.Conv2d(pam_channels, pam_channels // 4, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(pam_channels // 4),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1), # Optional dropout
            nn.Conv2d(pam_channels // 4, classes, kernel_size=1)
            # Final activation (Sigmoid for binary change) will be applied outside or in loss
        )

    def forward(self, x1, x2):
        # --- 1. Siamese Feature Extraction ---
        # Pass both images through the SHARED feature extractor
        feat_t1 = self.feature_extractor(x1) # X(1) in diagram
        feat_t2 = self.feature_extractor(x2) # X(2) in diagram
        
        # --- 2. Spatial-Temporal Attention ---
        # Apply PAM to both feature maps separately? Or concatenated?
        # The diagram seems to show applying attention *after* some implicit fusion?
        # Let's assume the standard way: apply attention to refine each map
        # *then* compare. OR, apply PAM to the *difference* or *concatenation*.
        # The diagram shows PAM applied to *both* X(1) and X(2) leading to Z(1) and Z(2).
        
        z1 = self.pam(feat_t1) # Z(1) in diagram
        z2 = self.pam(feat_t2) # Z(2) in diagram

        # --- 3. Final Comparison (Implicit in Metric Module) ---
        # How are Z(1) and Z(2) compared? Most likely subtraction or concatenation
        # Let's use subtraction as it's common for change detection
        feat_diff = torch.abs(z1 - z2)
        
        # --- 4. Metric Module (Classifier) ---
        change_map_logits = self.metric_module(feat_diff)
        
        # --- 5. Upsample to Original Size ---
        # The features are at 1/4 resolution, need to upsample
        change_map_logits = F.interpolate(change_map_logits, scale_factor=4, 
                                          mode='bilinear', align_corners=False)
                                          
        return change_map_logits

In [5]:
model = STANet(backbone_name='resnet18', pretrained=True, in_channels=3, classes=1)

In [6]:
print(model)

STANet(
  (feature_extractor): ResNetFPNFeatureExtractor(
    (backbone): FeatureListNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act1): ReLU(inplace=True)
          (aa): Identity()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): ReLU(inplace=True)
        )
        (1): 